In [67]:
import numpy as np
import pandas as pd
import tensorflow
from konlpy.tag import Kkma
from konlpy.tag import Komoran
from konlpy.tag import Okt
from gensim.models import Word2Vec
import time

In [23]:
# konlpy -> 형태소 분석기 3개
# Okt, kkma, komoran

In [24]:
# 단어 토큰화, 품사 태깅
# 형태소 : 의미가 있는 가장 작은 말의 단위
# 형태소를 토큰 단위로 사용해서 단어와 품사 정보를 사용

In [25]:
# 영어 -> 띄어쓰기를 기준으로 토큰화 시킬 수 있는데 한글은 불가능

# 지원하는 품사 갯수 : kkma > komoran > okt
# 정확도 : kkma > komoran > okt
# 속도 : kkma < komoran < okt

In [26]:
kkma = Kkma()

In [27]:
# 형태소 추출
text = '철수는 오렌지 하나를 사왔다'

morph = kkma.morphs(text)
morph

['철수', '는', '오렌지', '하나', '를', '사', '아', '오', '았', '다']

In [28]:
# 형태소 품사 태깅
pos = kkma.pos(text)
pos

# NNG -> 일반명사

[('철수', 'NNG'),
 ('는', 'JX'),
 ('오렌지', 'NNG'),
 ('하나', 'NNG'),
 ('를', 'JKO'),
 ('사', 'VV'),
 ('아', 'ECS'),
 ('오', 'VX'),
 ('았', 'EPT'),
 ('다', 'EFN')]

In [29]:
nouns = kkma.nouns(text)
nouns

['철수', '오렌지', '하나']

In [30]:
# 복합문장 분리
text = '철수는 오렌지 하나를 사왔다. 영희는 사과를 두개 사왔다'
sentences = kkma.sentences(text)
sentences

['철수는 오렌지 하나를 사 왔다.', '영희는 사과를 두개 사 왔다']

In [44]:
# komoran
Komoran = Komoran()
text = '철수는 오렌지 하나를 사왔다.'

morph = Komoran.morphs(text)
pos = Komoran.pos(text)
nouns = Komoran.nouns(text)

morph, pos, nouns

(['철수', '는', '오렌지', '하나', '를', '사', '아', '오', '았', '다', '.'],
 [('철수', 'NNP'),
  ('는', 'JX'),
  ('오렌지', 'NNP'),
  ('하나', 'NR'),
  ('를', 'JKO'),
  ('사', 'VV'),
  ('아', 'EC'),
  ('오', 'VX'),
  ('았', 'EP'),
  ('다', 'EF'),
  ('.', 'SF')],
 ['철수', '오렌지'])

In [32]:
# okt
Okt = Okt()

text = '철수는 오렌지 하나를 사왔다.'

morph = Okt.morphs(text)
pos = Okt.pos(text)
nouns = Okt.nouns(text)

morph, pos, nouns

(['철수', '는', '오렌지', '하나', '를', '사왔다', '.'],
 [('철수', 'Noun'),
  ('는', 'Josa'),
  ('오렌지', 'Noun'),
  ('하나', 'Noun'),
  ('를', 'Josa'),
  ('사왔다', 'Verb'),
  ('.', 'Punctuation')],
 ['철수', '오렌지', '하나'])

In [35]:
# Okt 지원함수 두개

# 구어체
text = '오늘 날씨가 좋아욬ㅋㅋㅋ'
# 정규화 (구어체, 오타 잡아줌)
print(Okt.normalize(text))

# 어구추출
print(Okt.phrases(text))


오늘 날씨가 좋아요ㅋㅋㅋ
['오늘', '오늘 날씨', '좋아욬', '날씨']


In [36]:
text = "나는 영화 나는 내일, 어제의 너와 만난다를 볼거야"

pos = Komoran.pos(text)
pos

[('나', 'NP'),
 ('는', 'JX'),
 ('영화', 'NNG'),
 ('나', 'NP'),
 ('는', 'JX'),
 ('내일', 'NNG'),
 (',', 'SP'),
 ('어제', 'NNP'),
 ('의', 'JKG'),
 ('너', 'NP'),
 ('와', 'JKB'),
 ('만나', 'VV'),
 ('ㄴ다', 'EC'),
 ('를', 'JKO'),
 ('보', 'VV'),
 ('ㄹ', 'ETM'),
 ('거', 'NNB'),
 ('야', 'JX')]

In [48]:
# 사용자 사전 -> 단어 추가
Komoran = Komoran(userdic="./data/user_dic.tsv")

text = "나는 영화 나는 내일, 어제의 너와 만난다를 볼거야"

pos = Komoran.pos(text)
pos

[('나', 'NP'),
 ('는', 'JX'),
 ('영화', 'NNG'),
 ('나는 내일, 어제의 너와 만난다', 'NNG'),
 ('를', 'JKO'),
 ('보', 'VV'),
 ('ㄹ', 'ETM'),
 ('거', 'NNB'),
 ('야', 'JX')]

In [51]:
# 단어의 의미를 분석해서 실수 벡터로 바꿔주는 기법

komoran = Komoran()
text = '오늘 날씨는 구름이 많아요'

In [52]:
# 원 핫 인코딩

nouns = komoran.nouns(text)
nouns

['오늘', '날씨', '구름']

In [54]:
# [1,0,0] -> 오늘
# [0,1,0] -> 날씨
# [0,0,1] -> 구름

# 단어 사전 구축, 단어별 인덱스 부여

dic = {}
for word in nouns:
    if word not in dic.keys():
        dic[word] = len(dic)

dic
# key : 단어, value : 인덱스

{'오늘': 0, '날씨': 1, '구름': 2}

In [55]:
nb = len(dic)
target = list(dic.values())
# np.eye -> 단위행렬
# 단어 사전의 길이만큼 단위행렬
oh_target = np.eye(nb)[target]
oh_target

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

In [56]:
# 단어사전의 크기가 커지면 원 핫 인코딩을 사용했을 떄, 메모리도 낭비되고 훈련시간 길어짐
# -> 단어 임베딩

In [ ]:
# 구글 -> word2vec
# gensim

# word2vec 두가지 모델
# 1. CBOW -> 주변 단어를 통해서 타깃 단어를 예측
# 2. skip-gram -> 타깃 단어를 통해서 주변 단어를 예측
# 오늘 날씨는 구름이 많아요
# CBOW
# 오늘 __는 구름이 많아요
# 타깃 단어 : 날씨, 주변단어 오늘, 구름

# skip-gram
# __ 날씨는 __이 많아요
# 주변 단어 : 오늘, 구름 -> 타깃 단어 : 날씨

# window : 주변 단어의 갯수
# window가 1이면 타깃 단어 앞뒤로 1개씩 주변단어로 보는 것


In [60]:
# 데이터 읽어오기

def read_review_data(filename):
    with open(filename, 'r', encoding='utf-8') as f :
        # f.read().splitlines() 줄별로 잘라서 읽은 것
        # line.split('\t') 줄별로 자른거 -> 탭 기준으로 문장을 자른 것
        data = [line.split('\t') for line in f.read().splitlines()]
        data = data[1:]
    return data

In [68]:
# 수행속도 확인
start = time.time()

# 파일 읽어오기
review_data = read_review_data("./data/ratings.txt")
print(len(review_data))
print("1. 파일 읽어오기 완료 : ", time.time() - start)

# 문장별로 명사만 추출해서 입력 데이터 만들기
Komoran = Komoran()
docs = [Komoran.nouns(sentences[1]) for sentences in review_data]
print("2. 명사 추출 완료 : ", time.time() - start)

# word2vec 모델 훈련
# sentences : 문장데이터
# vector_size : 단어 임베딩 된 벡터의 크기
# window : 주변 단어의 갯수
# min_count : 단어 최소 빈도수 (빈도수 이하의 단어는 카운트 안됨)
# sg : 0(CBOW) / 1(skip-gram)
model = Word2Vec(sentences=docs, vector_size=200, window=4, min_count=2, sg=1)
print("3. word2vec 모델 학습 완료 : ", time.time() - start)

# 모델 저장
model.save("embedding.model")
print("4. 모델 저장 완료 : ", time.time() - start)

# 리뷰 갯수
print(model.corpus_count)

# 학습한 단어 갯수
print(model.corpus_total_words)

200000
1. 파일 읽어오기 완료 :  0.40199756622314453
2. 명사 추출 완료 :  225.29833984375
3. word2vec 모델 학습 완료 :  237.06633305549622
4. 모델 저장 완료 :  237.1043348312378
200000
1076896


In [78]:
# 저장한 모델 이용해서 확인
model = Word2Vec.load('./embedding.model')

# 단어 임베딩 벡터
model.wv

In [77]:
model.wv.similarity(w1='일요일', w2='월요일')

0.91994405

In [74]:
model.wv.similarity(w1='안성기', w2='배우')

0.7024495

In [75]:
model.wv.similarity(w1='일요일', w2='배우')

0.4075816

In [72]:
# 가장 유사한 단어
model.wv.most_similar("안성기", topn=5)

[('양동근', 0.946983814239502),
 ('박신양', 0.9388031363487244),
 ('정려원', 0.9386017322540283),
 ('재발견', 0.9378085136413574),
 ('심은하', 0.9377965331077576)]

In [73]:
model.wv.most_similar("장르", topn=5)

[('영화 장르', 0.8765575289726257),
 ('물이', 0.8629868626594543),
 ('분류', 0.8422165513038635),
 ('정통', 0.8385854959487915),
 ('혼합', 0.8363239765167236)]